In [3]:
import os
import csv
from supabase import create_client, Client
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path('..') / '.env.local'
load_dotenv(dotenv_path=env_path)

url = os.getenv("NEXT_PUBLIC_SUPABASE_URL")
key = os.getenv("NEXT_PUBLIC_SUPABASE_ANON_KEY")
supabase: Client = create_client(url, key)

data = supabase.table("league").select("*").execute()
print(data)

data=[{'league_unique': 'ncaambb', 'league_name': "NCAA Men's Basketball", 'sport': 'basketball', 'womens': False}] count=None


In [13]:
def update_game_schedule(game_schedule_csv):
    game_insert = []
    with open(game_schedule_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        next(reader)
        for row in reader:
            game_insert.append(row)

    data = supabase.table("game").upsert(game_insert,
                                         # ignore_duplicates=True, on_conflict="game_date, team_1_id, team_2_id"
                                         ).execute()
    print(data)

In [14]:
update_game_schedule("game-schedule-2023-ncaa-tournament.csv")

data=[{'game_id': 1, 'game_date': '2023-03-16', 'team_1_id': 'virginia', 'team_2_id': 'furman', 'game_time': '12:40:00', 'round_num': 2, 'competition_id': 1}, {'game_id': 2, 'game_date': '2023-03-16', 'team_1_id': 'missouri', 'team_2_id': 'utah-state', 'game_time': '13:40:00', 'round_num': 2, 'competition_id': 1}, {'game_id': 3, 'game_date': '2023-03-16', 'team_1_id': 'kansas', 'team_2_id': 'howard', 'game_time': '14:00:00', 'round_num': 2, 'competition_id': 1}, {'game_id': 4, 'game_date': '2023-03-16', 'team_1_id': 'alabama', 'team_2_id': 'texas-am-corpus-christi', 'game_time': '14:45:00', 'round_num': 2, 'competition_id': 1}, {'game_id': 5, 'game_date': '2023-03-16', 'team_1_id': 'san-diego-state', 'team_2_id': 'college-of-charleston', 'game_time': '15:10:00', 'round_num': 2, 'competition_id': 1}, {'game_id': 6, 'game_date': '2023-03-16', 'team_1_id': 'arizona', 'team_2_id': 'princeton', 'game_time': '16:10:00', 'round_num': 2, 'competition_id': 1}, {'game_id': 7, 'game_date': '2023-

In [6]:
import pandas as pd
import datetime
def generate_game_scoring_sheet(competition_id: int = 1, date = datetime.date.today()):
    data = supabase.table("players_in_games_view").select("*").eq("game_date", date).eq('competition_id', competition_id).execute()
    df = pd.DataFrame(data.data)
    df['points'] = None
    # df['rebounds'] = 0
    # df['assists'] = 0
    df.to_csv(f"{date}-game-scoring-2023-ncaa-tournament.csv", index=False, columns=['game_time', 'team_unique', 'player_unique', 'points','game_id'])
    # 'rebounds', 'assists'])

In [5]:
generate_game_scoring_sheet()

KeyError: "['game_time', 'team_unique', 'player_unique', 'game_id'] not in index"

In [15]:
def update_scores_from_csv(current_round: int, competition_id: int = 1, game_scoring_csv = f"{datetime.date.today()}-game-scoring-2023-ncaa-tournament.csv"):
    player_game_insert = []
    with open(game_scoring_csv, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        next(reader)
        for row in reader:
            if row['points'] != '':
                player_game_row = {
                    "player_unique": row['player_unique'],
                    "game_id": row['game_id'],
                    "points": row['points'],
                }
                player_game_insert.append(player_game_row)

    data = supabase.table("player_game").upsert(player_game_insert,
                                         ignore_duplicates=False, on_conflict="game_id, player_unique").execute()
    updated = supabase.table("competition_updated").insert({"competition_id": competition_id, "current_round": current_round}).execute()
    print(data)

In [18]:
update_scores_from_csv(2)

data=[{'player_unique': 'joey-hauser-1', 'game_id': 16, 'points': 17, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'aj-hoggard-1', 'game_id': 16, 'points': 11, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'jaden-akins-1', 'game_id': 16, 'points': 12, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'malik-hall-1', 'game_id': 16, 'points': 6, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'mady-sissoko-1', 'game_id': 16, 'points': 4, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'pierre-brooks-1', 'game_id': 16, 'points': 0, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'jaxon-kohler-1', 'game_id': 16, 'points': 2, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'tre-holloman-1', 'game_id': 16, 'points': 2, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'carson-cooper-1', 'game_id': 16, 'points': 6, 'rebounds': 0, 'assists': 0, 'blocks': 0}, {'player_unique': 'jason-whitens